In [18]:
PROJECT_HOME='/kaggle'

# UPDATE THE CODE BELOW BEFORE RUNNING LOCALLY
# PROJECT_HOME='/Users/athul/git/map-kaggle'

In [2]:
import importlib
import numpy as np

#Scikit-learn
from sklearn.model_selection import train_test_split

# Keras
from keras.utils.np_utils import to_categorical
from keras import backend as K

# Tensorflow
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [3]:
# baseline

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

RANDOM_STATE = 0

# baseline - SVM
def train_svm(X_train, y_train, random_state=RANDOM_STATE, verbosity=True):
    clf = SVC(random_state=random_state, verbose=verbosity)
    clf.fit(X_train_flattened, y_train)
    return clf

# baseline - logistic regression
def train_logistic_regression(X_train, y_train, random_state=RANDOM_STATE, max_iter=100, verbosity=1):
    hyperparameters = []
    clf = LogisticRegression(max_iter=max_iter, verbose=verbosity, random_state=random_state)
    clf.fit(X_train, y_train)
    return clf

In [4]:
# data augmentation

import numpy as np
import pandas as pd
import os
import tensorflow as tf

def add_gaussian_noise(X_imgs):
    gaussian_noise_imgs = []    
    for X_img in X_imgs:
        noise = tf.random.normal(shape=tf.shape(X_img), mean=0.0, stddev=0.1, dtype=tf.float32)
        noise_img = X_img + noise
        noise_img = tf.clip_by_value(noise_img, 0.0, 1.0)
        gaussian_noise_imgs.append(noise_img)
    gaussian_noise_imgs = np.array(gaussian_noise_imgs, dtype = np.float32)
    return gaussian_noise_imgs

def rotate(X_imgs, rotation_range=90):
    rotated_imgs = []
    for X_img in X_imgs:
        rotated_img = tf.keras.image.random_rotation(
            X_img, rotation_range, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest',
            cval=0.0, interpolation_order=1)
        rotated_imgs.append(rotated_img)
    return rotated_imgs

def zoom(X_imgs, zoom_range=(0.5,1.3)):
    zoomed_imgs = []
    for X_img in X_imgs:
        zoomed_img = tf.keras.image.random_zoom(
            X_img, zoom_range, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
        zoomed_imgs.append(zoomed_img)
    return zoomed_imgs

def horizontal_flip(X_imgs):
    flipped_imgs = []
    for X_img in X_imgs:
        flipped_img = tf.image.flip_left_right(X_img)
        flipped_imgs.append(X_img)
    return flipped_imgs
  
def vertical_flip(X_imgs):
    flipped_imgs = []
    for X_img in X_imgs:
        flipped_img = tf.image.flip_up_down(X_img)
        flipped_imgs.append(X_img)
    return flipped_imgs


def selective_augment(X_imgs, y_labels, times=2):
    augmented_data_X = np.concatenate((X_imgs, horizontal_flip(X_imgs)))
    augmented_data_Y = np.concatenate((y_labels, y_labels))
    if times==3: # add more samples to triple the dataset 
        vertical_flipped_imgs = vertical_flip(X_imgs)
        augmented_data_X = np.concatenate((augmented_data_X, vertical_flipped_imgs))
        augmented_data_Y = np.concatenate((augmented_data_Y, y_labels))
    return (augmented_data_X, augmented_data_Y)

def get_image_data_generator(rotation_range=30, zoom_range=[0.8, 1.2], brightness_range=[0.8,1.0]):
    data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
		rotation_range=rotation_range, zoom_range=zoom_range,
		brightness_range=brightness_range)
    print('Returning image data generator')
    return data_generator

In [5]:
# metrics

from keras import backend as K
from sklearn.metrics import f1_score
import numpy as np

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1_micro(y_true, y_pred):
	y_pred_flattened = np.argmax(y_pred, axis=1)
	y_true_flattened = np.argmax(y_true, axis=1)
	return f1_score(y_true_flattened, y_pred_flattened, average='micro')

In [6]:
# CNN models

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from keras.layers import BatchNormalization


def m1():
    model = Sequential([
    Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(96,96,1),padding='same'),
    # BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D((2, 2),padding='same'),

    Conv2D(64, (3, 3), activation='linear',padding='same',kernel_regularizer=l2(0.01)),
    # BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2, 2),padding='same'),

    Conv2D(128, (3, 3), activation='linear',padding='same'),
    # BatchNormalization(),
    LeakyReLU(alpha=0.1),          
    MaxPooling2D(pool_size=(2, 2),padding='same'),

    Conv2D(256, (3, 3), activation='linear',padding='same',kernel_regularizer=l2(0.01)),
    # BatchNormalization(),
    LeakyReLU(alpha=0.1),                 
    MaxPooling2D(pool_size=(2, 2),padding='same'),
    Dropout(0.25),

    Flatten(),

    Dense(128, activation='linear'),
    # BatchNormalization(),
    LeakyReLU(alpha=0.1),  
    Dropout(0.25),

    Dense(num_classes, activation='softmax')])
    model.compile(optimizer =  Adam() , loss = "categorical_crossentropy", metrics=["accuracy"])
    return model

def m2(num_classes=11):
    model_master = Sequential()

    model_master.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(96,96,1), padding="same"))
    model_master.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
    model_master.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
    model_master.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model_master.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
    model_master.add(Flatten())
    model_master.add(Dense(84, activation='relu'))
    model_master.add(Dense(num_classes, activation='softmax'))
    return model_master

def m3():
    model_master = Sequential()
    model_master.add(BatchNormalization())

    model_master.add(Conv2D(96, kernel_size=(3, 3), activation='relu', input_shape=(96,96,1), kernel_initializer='he_normal', padding="same"))
    model_master.add(Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same'))
    model_master.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))
    model_master.add(BatchNormalization())
    model_master.add(MaxPooling2D(pool_size=(3, 3), strides = 2))
	
    model_master.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model_master.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model_master.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model_master.add(BatchNormalization())
    model_master.add(MaxPooling2D(pool_size=(3, 3), strides = 2))

    model_master.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model_master.add(Conv2D(192, (1, 1), activation='relu'))
    model_master.add(Conv2D(11, (1, 1)))
    model_master.add(BatchNormalization())
    model_master.add(GlobalAveragePooling2D())

    model_master.add(Activation(activation='softmax'))
    return model_master

def m4():
    model = Sequential()
    
    model.add(Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same',input_shape=(96,96,1)))
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (1, 1), activation='relu'))
    model.add(Conv2D(11, (1, 1)))
    model.add(GlobalAveragePooling2D())
    model.add(Activation(activation='softmax'))
    return model

def m5() :
    model = Sequential()

    #mlpconv block 1
    model.add(Conv2D(32, (5, 5), activation='relu',padding='valid', input_shape = (96,96,1)))
    model.add(Conv2D(32, (1, 1), activation='relu'))
    model.add(Conv2D(32, (1, 1), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.5))
    
    #mlpconv block2
    model.add(Conv2D(64, (3, 3), activation='relu',padding='valid'))
    model.add(Conv2D(64, (1, 1), activation='relu'))
    model.add(Conv2D(64, (1, 1), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.5))
    
    #mlpconv block3
    model.add(Conv2D(128, (3, 3), activation='relu',padding='valid'))
    model.add(Conv2D(32, (1, 1), activation='relu'))
    model.add(Conv2D(11, (1, 1)))
    
    model.add(GlobalAveragePooling2D())
    model.add(Activation(activation='softmax'))    
    return model



# CNN
def create_cnn(num_classes):
    model = Sequential([
        
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(96,96,1), padding='same'),
        LeakyReLU(alpha=0.1),
        MaxPooling2D((2, 2), padding='same'),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        LeakyReLU(alpha=0.1),
        MaxPooling2D(pool_size=(2, 2), padding='same'),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        LeakyReLU(alpha=0.1),          
        MaxPooling2D(pool_size=(2, 2), padding='same'),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        LeakyReLU(alpha=0.1),                 
        MaxPooling2D(pool_size=(2, 2), padding='same'),
        Dropout(0.2),

        Flatten(),

        Dense(128, activation='relu'),
        LeakyReLU(alpha=0.1),  
        Dropout(0.2),

        Dense(num_classes, activation='softmax')
    ])
    return model

def simple_cnn(input_shape=(96, 96, 1), num_classes=11):
	
	# building a linear stack of layers with the sequential model
	model = Sequential()
	# convolutional layer
	model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=input_shape))
	model.add(MaxPool2D(pool_size=(1,1)))
	# flatten output of conv
	model.add(Flatten())
	# hidden layer
	model.add(Dense(100, activation='relu'))
	# output layer
	model.add(Dense(num_classes, activation='softmax'))
	return model

def simple_cnn2(input_shape=(96, 96, 1), num_classes=11):
	
	# building a linear stack of layers with the sequential model
	model = Sequential()

	# convolutional layer
	model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=input_shape))

	# convolutional layer
	model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
	model.add(MaxPool2D(pool_size=(2,2)))
	model.add(Dropout(0.25))

	model.add(Conv2D(250, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
	model.add(MaxPool2D(pool_size=(2,2)))
	model.add(Dropout(0.25))

	# flatten output of conv
	model.add(Flatten())

	# hidden layer
	model.add(Dense(500, activation='relu'))
	model.add(Dropout(0.4))
	model.add(Dense(250, activation='relu'))
	model.add(Dropout(0.3))
	# output layer
	model.add(Dense(num_classes, activation='softmax'))
	
	return model

def resnet50(input_shape, num_classes=11):
    return tf.keras.applications.ResNet50V2(weights=None,
        input_shape=input_shape, pooling=None, classes=num_classes)

In [7]:
# preprocessing

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from keras.utils.np_utils import to_categorical

def encode(y):
    le = LabelEncoder()
    le.fit(y)
    return le.transform(y)

def encode_to_categorical(y, num_classes=11):
	y_encoded = encode(y)
	y_encoded_categorical = to_categorical(y_encoded, num_classes)
	return y_encoded_categorical

def flatten(dataframe):
    return dataframe.reshape(dataframe.shape[0], dataframe.shape[1] * dataframe.shape[2])

def balance(X, y):
    augmented_data_X = np.empty((0, X.shape[1], X.shape[2], X.shape[3]))
    augmented_data_Y = []
    freq = np.unique(y, return_counts=True)
    for index in range(len(freq[0])):
        label_index = np.where(y == freq[0][index])
        if freq[1][index] < 700:
            augmented_data = selective_augment(X[label_index], y[label_index], 3)
        elif freq[1][index] < 1000:
            augmented_data = selective_augment(X[label_index], y[label_index])
        else:
            augmented_data = (X[label_index], y[label_index])

        augmented_data_X = np.concatenate((augmented_data_X, augmented_data[0]))
        augmented_data_Y = np.concatenate((augmented_data_Y, augmented_data[1]))
    return augmented_data_X, augmented_data_Y

In [8]:
# utils

import pandas as pd
import numpy as np
import time
import csv

SUBMISSION_HEADER = ['Id', 'class']

def open_pickled_file(file):
    with open(file, 'rb') as f:
        data = pd.read_pickle(file)
    return np.asarray(data)

def export_predictions(y_pred):
    timestr = time.strftime("%Y%m%d-%H%M%S")
    filename = PROJECT_HOME + '/output/prediction-{}.csv'.format(timestr)
    file = open(filename, 'w+', newline ='')
    with file:    
        writer = csv.writer(file)
        writer.writerow(SUBMISSION_HEADER)
        writer.writerows(enumerate(y_pred))
    print('Predictions exported to {}'.format(filename))
	
def show_image(data):
    plt.imshow(data, interpolation='nearest')
    plt.show()

def check_dataset(X, y, count=1):
    randomlist = random.sample(range(len(X)), count)
    for i in randomlist:
        show_image(X[i])
        print(y[i])
		
def predict_and_export(clf, X):
    print('Predicting..')
    y_pred = clf.predict(X)
    print('Exporting predictions..')
    export_predictions(y_pred)
    print('Export completed')  

In [9]:
# visualizations

import matplotlib.pyplot as plt

def plot_accuracy(model):
	# accuracy of our model
	plt.figure(figsize=(12, 6))
	plt.plot(model.history["accuracy"])
	plt.plot(model.history["val_accuracy"])
	plt.xlabel("Epochs")
	plt.ylabel("Accuracy")
	plt.title("ACCURACY OF MODEL")
	plt.legend(['training_accuracy', 'validation_accuracy'])
	plt.show()
	
def plot_loss(model):
	# loss of our model
	plt.figure(figsize=(12, 6))
	plt.plot(model.history["loss"])
	plt.plot(model.history["val_loss"])
	plt.xlabel("Epochs")
	plt.ylabel("Loss")
	plt.title("LOSS OF MODEL")
	plt.legend(['training_loss', 'validation_loss'])
	plt.show()

In [10]:
def split_dataset(X_dataset, y_dataset, test_size=0.2, random_state=0, shuffle=False):
    return train_test_split(X_dataset, y_dataset, test_size=test_size, random_state=random_state, shuffle=shuffle)

In [19]:
X_train_raw = open_pickled_file(PROJECT_HOME + '/input/fall2021-inf8245e-machine-learning/x_train.pkl')
y_train_raw = open_pickled_file(PROJECT_HOME + '/input/fall2021-inf8245e-machine-learning/y_train.pkl')
X_test_raw = open_pickled_file(PROJECT_HOME + '/input/fall2021-inf8245e-machine-learning/x_test.pkl')

In [20]:
# print(X_train_raw.shape)
# print(y_train_raw.shape)
# print(X_test_raw.shape)
# freq = np.unique(y_train_raw, return_counts=True)
# print(freq)

In [21]:
#scale and flatten X

X_train_scaled = X_train_raw/255
X_test_scaled = X_test_raw/255

X_train_scaled_reshaped = X_train_scaled.reshape(-1, 96, 96, 1)
X_test_scaled_reshaped = X_test_scaled.reshape(-1, 96, 96, 1)

# Required to train baseline models
# X_train_scaled_flattened = flatten(X_train_scaled)
# X_test_scaled_flattened = flatten(X_test_scaled)

#encode y_train
y_train_encoded = encode(y_train_raw)
Y_train_encoded_categorical = to_categorical(y_train_encoded, 11)

print("Balancing data...")
balanced_dataset = balance(X_train_scaled_reshaped, y_train_encoded)

X_train, X_val, y_train, y_val = split_dataset(balanced_dataset[0], to_categorical(balanced_dataset[1], 11), shuffle=True)

Balancing data...


In [22]:
# Uncomment the below section to train baseline models. Make sure the inputs are flattened

# print('Training Logistic regression')
# clf = train_loy_predc_regression(X_train, y_train)
# predict_and_export(clf, X_test_flattened)

# print('Training SVM')
# clf = train_svm(X_train, y_train)
# predict_and_export(clf, X_test_flattened)

In [23]:
TRAIN_BATCH_SIZE=16
image_data_generator = get_image_data_generator()

Returning image data generator


In [24]:
print('Training CNN')
tf.test.is_built_with_cuda()
tf.test.gpu_device_name()

initial_learning_rate = 0.0001
epochs = 100

filepath = './weights/' + '.{epoch:02d}-{loss:.2f}.hdf5'
mcp_save = ModelCheckpoint(filepath, monitor='val_f1_m', mode='max')

early_stopping_callback = EarlyStopping(
    monitor='val_f1_m', 
    min_delta=0.001,
    patience=3)

reduce_lr_callback = ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.1,
    patience=3,
    verbose=0,
    mode="max",
    min_delta=0.01,
    cooldown=0,
    min_lr=0
)

opt = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)

model = m3()
image_data_generator = get_image_data_generator(rotation_range=30, zoom_range=[0.9, 1.1], brightness_range=[0.8, 1])

with tf.device('/GPU:0'):
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy', f1_m])
    Model = model.fit(
        image_data_generator.flow(X_train, y_train, shuffle=True, batch_size=TRAIN_BATCH_SIZE),
        validation_data=image_data_generator.flow(X_val, y_val, shuffle=True, batch_size=TRAIN_BATCH_SIZE), 
        epochs=epochs, 
        shuffle=True, 
        callbacks=[reduce_lr_callback, mcp_save], 
        batch_size=TRAIN_BATCH_SIZE)

    plot_accuracy(Model)
    plot_loss(Model)
    
    y_pred = model.predict(X_val)
    f1_s = f1_micro(y_val, y_pred)
    print("Validation f1-score:", f1_s)

Training CNN
Returning image data generator


2021-12-12 17:12:46.376355: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-12 17:12:46.376528: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-12-12 17:12:48.789208: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-12 17:12:48.800523: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2021-12-12 17:12:49.325416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


847/847 [==============================] - ETA: 0s - loss: 2.0818 - accuracy: 0.3167 - f1_m: 0.0056

2021-12-12 17:17:08.679445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


847/847 [==============================] - 275s 323ms/step - loss: 2.0818 - accuracy: 0.3167 - f1_m: 0.0056 - val_loss: 2.0822 - val_accuracy: 0.2853 - val_f1_m: 0.0142
Epoch 2/100
730/847 [========================>.....] - ETA: 36s - loss: 1.9204 - accuracy: 0.4177 - f1_m: 0.0226

KeyboardInterrupt: 

In [25]:
# Predict and export the results on test data
pred = model.predict(X_test_scaled_reshaped)
pred = np.argmax(pred, axis=1)
export_predictions(pred)

2021-12-12 17:21:19.443133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Predictions exported to /Users/athul/git/map-kaggle/output/prediction-20211212-172240.csv
